# HuggingFace Embeddings

이 노트북에서는 LangChain에서 HuggingFace의 임베딩 모델을 활용하는 방법을 학습합니다.

## 학습 내용
- HuggingFace 임베딩 모델 설정
- 텍스트를 벡터로 변환하는 방법
- 다양한 임베딩 모델 활용법

## 1. 환경 설정

먼저 필요한 라이브러리를 설치하고 API 키를 설정합니다.

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

## 2. HuggingFace Embeddings 기본 사용법

### 2.1 필요한 패키지 임포트

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings as HFEmbeddings

### 2.2 기본 임베딩 모델 초기화

HuggingFace의 기본 임베딩 모델을 사용하여 텍스트를 벡터로 변환합니다.

In [ ]:
# 기본 모델 사용 (sentence-transformers/all-MiniLM-L6-v2)
embeddings = HuggingFaceEmbeddings()

# 텍스트를 임베딩으로 변환
text = "안녕하세요, HuggingFace 임베딩을 학습하고 있습니다."
query_result = embeddings.embed_query(text)

# 임베딩 벡터의 차원 확인
print(f"임베딩 벡터 차원: {len(query_result)}")
print(f"임베딩 벡터 일부: {query_result[:5]}")

### 2.3 특정 모델 지정하기

다양한 HuggingFace 임베딩 모델을 지정하여 사용할 수 있습니다.

In [ ]:
# 한국어에 특화된 모델 사용
model_name = "jhgan/ko-sroberta-multitask"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}

hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# 한국어 텍스트 임베딩
korean_text = "자연어 처리는 인공지능의 중요한 분야입니다."
korean_embedding = hf_embeddings.embed_query(korean_text)

print(f"한국어 임베딩 차원: {len(korean_embedding)}")

## 3. 여러 문서 임베딩

여러 문서를 한 번에 임베딩하는 방법을 알아봅니다.

In [ ]:
# 여러 문서 준비
documents = [
    "LangChain은 LLM 애플리케이션 개발을 위한 프레임워크입니다.",
    "HuggingFace는 다양한 사전 학습된 모델을 제공합니다.",
    "임베딩은 텍스트를 숫자 벡터로 변환하는 과정입니다.",
    "벡터 데이터베이스는 임베딩을 효율적으로 저장하고 검색합니다."
]

# 문서들을 임베딩으로 변환
doc_embeddings = hf_embeddings.embed_documents(documents)

# 결과 확인
print(f"임베딩된 문서 수: {len(doc_embeddings)}")
print(f"각 문서의 임베딩 차원: {len(doc_embeddings[0])}")

## 4. 임베딩 유사도 계산

생성된 임베딩 벡터 간의 유사도를 계산하여 문서 간 관련성을 파악할 수 있습니다.

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 쿼리와 문서들 간의 유사도 계산
query = "LLM 프레임워크에 대해 알려주세요."
query_embedding = hf_embeddings.embed_query(query)

# 코사인 유사도 계산
similarities = cosine_similarity([query_embedding], doc_embeddings)[0]

# 결과 출력
for i, (doc, sim) in enumerate(zip(documents, similarities)):
    print(f"문서 {i+1} (유사도: {sim:.4f}): {doc}")

# 가장 유사한 문서 찾기
most_similar_idx = np.argmax(similarities)
print(f"\n가장 유사한 문서: {documents[most_similar_idx]}")

## 5. 다양한 임베딩 모델 비교

여러 임베딩 모델의 성능과 특징을 비교해봅니다.

In [ ]:
# 비교할 모델들
models = [
    "sentence-transformers/all-MiniLM-L6-v2",  # 영어 범용
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",  # 다국어
    "jhgan/ko-sroberta-multitask"  # 한국어 특화
]

test_texts = [
    "Machine learning is fascinating.",
    "기계 학습은 매우 흥미롭습니다.",
    "機械学習は魅力的です。"
]

# 각 모델로 임베딩 생성 및 비교
for model_name in models:
    print(f"\n모델: {model_name}")
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    
    for text in test_texts:
        try:
            embedding = embeddings.embed_query(text)
            print(f"  텍스트: {text[:30]}... -> 차원: {len(embedding)}")
        except Exception as e:
            print(f"  텍스트: {text[:30]}... -> 오류: {str(e)}")

## 6. 캐싱을 통한 성능 최적화

임베딩 결과를 캐싱하여 동일한 텍스트에 대한 반복 계산을 줄일 수 있습니다.

In [ ]:
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
import time

# 캐시 저장소 설정
store = LocalFileStore("./cache/")

# 기본 임베딩 모델
underlying_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# 캐시 백엔드 임베딩 생성
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings, store, namespace=underlying_embeddings.model_name
)

# 성능 테스트
test_text = "이것은 캐싱 테스트를 위한 문장입니다."

# 첫 번째 실행 (캐시 없음)
start_time = time.time()
embedding1 = cached_embeddings.embed_query(test_text)
first_time = time.time() - start_time

# 두 번째 실행 (캐시 사용)
start_time = time.time()
embedding2 = cached_embeddings.embed_query(test_text)
second_time = time.time() - start_time

print(f"첫 번째 실행 시간: {first_time:.4f}초")
print(f"두 번째 실행 시간 (캐시): {second_time:.4f}초")
print(f"속도 향상: {first_time/second_time:.2f}배")

## 7. 실제 활용 예제: 문서 검색 시스템

HuggingFace 임베딩을 활용한 간단한 문서 검색 시스템을 구현해봅니다.

In [ ]:
class SimpleDocumentSearch:
    def __init__(self, model_name="jhgan/ko-sroberta-multitask"):
        self.embeddings = HuggingFaceEmbeddings(model_name=model_name)
        self.documents = []
        self.doc_embeddings = []
    
    def add_documents(self, documents):
        """문서를 추가하고 임베딩을 생성합니다."""
        self.documents.extend(documents)
        new_embeddings = self.embeddings.embed_documents(documents)
        self.doc_embeddings.extend(new_embeddings)
    
    def search(self, query, top_k=3):
        """쿼리와 가장 유사한 문서를 검색합니다."""
        query_embedding = self.embeddings.embed_query(query)
        
        # 유사도 계산
        similarities = cosine_similarity([query_embedding], self.doc_embeddings)[0]
        
        # 상위 k개 문서 선택
        top_indices = np.argsort(similarities)[::-1][:top_k]
        
        results = []
        for idx in top_indices:
            results.append({
                'document': self.documents[idx],
                'similarity': similarities[idx]
            })
        
        return results

# 문서 검색 시스템 사용 예제
search_system = SimpleDocumentSearch()

# 문서 추가
knowledge_base = [
    "Python은 배우기 쉽고 강력한 프로그래밍 언어입니다.",
    "JavaScript는 웹 개발에 필수적인 언어입니다.",
    "기계학습은 데이터에서 패턴을 찾는 기술입니다.",
    "딥러닝은 인공 신경망을 사용하는 기계학습의 한 분야입니다.",
    "자연어 처리는 컴퓨터가 인간의 언어를 이해하도록 하는 기술입니다."
]

search_system.add_documents(knowledge_base)

# 검색 수행
query = "인공지능과 관련된 기술을 알려주세요"
results = search_system.search(query, top_k=3)

print(f"쿼리: {query}\n")
print("검색 결과:")
for i, result in enumerate(results, 1):
    print(f"{i}. 유사도: {result['similarity']:.4f}")
    print(f"   문서: {result['document']}\n")

## 요약

이 노트북에서는 LangChain에서 HuggingFace 임베딩을 활용하는 방법을 학습했습니다:

1. **기본 사용법**: HuggingFaceEmbeddings 클래스를 사용한 텍스트 임베딩
2. **모델 선택**: 다양한 임베딩 모델 (영어, 한국어, 다국어) 활용
3. **문서 임베딩**: 여러 문서를 한 번에 임베딩하는 방법
4. **유사도 계산**: 코사인 유사도를 이용한 문서 관련성 측정
5. **성능 최적화**: 캐싱을 통한 임베딩 계산 최적화
6. **실제 활용**: 간단한 문서 검색 시스템 구현

HuggingFace 임베딩은 RAG(Retrieval-Augmented Generation) 시스템, 의미 기반 검색, 문서 분류 등 다양한 NLP 애플리케이션에서 핵심적인 역할을 합니다.